1) Carrega bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configura visualização
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_theme()

2) Carrega dataset

In [3]:
caminho_dados = 'dados/acidentes.csv'

try:
    df = pd.read_csv(caminho_dados, sep=';', encoding='utf-8')
    print("CSV original lido com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo '{caminho_dados}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

CSV original lido com sucesso!


3) Entende dataframe

In [4]:
# Lista as cinco primeira linhas
print("\nCinco primeiras linhas:")
print(df.head())

# Mostra o número de linhas e colunas
print("\nDimensões do Dataset (linhas, colunas)")
print(df.shape)

# Mostra tipo de dados, valores não nulos
print("\nInformações gerais do DataFrame:")
df.info()

# Conta valores ausentes por coluna
print("\nContagem de Valores Ausentes por Coluna")
print(df.isnull().sum())



Cinco primeiras linhas:
         data_extracao  predial1  queda_arr                 data  feridos  \
0  2025-06-01 01:33:13       0.0        0.0  2020-10-17 00:00:00        1   
1  2025-06-01 01:33:13     598.0        0.0  2020-01-01 00:00:00        1   
2  2025-06-01 01:33:13    1271.0        0.0  2020-01-01 00:00:00        1   
3  2025-06-01 01:33:13    1901.0        0.0  2020-01-02 00:00:00        2   
4  2025-06-01 01:33:13    3302.0        0.0  2020-01-02 00:00:00        1   

   feridos_gr  mortes  morte_post  fatais  auto  taxi  lotacao  onibus_urb  \
0           0       0           0       0     3     0        0           0   
1           0       0           0       0     0     1        0           0   
2           1       0           0       0     1     0        0           0   
3           0       0           0       0     0     0        0           0   
4           0       0           0       0     1     0        0           0   

   onibus_met  onibus_int  caminhao  moto  

4) Limpa nomes colunas

In [5]:
# Remove espaços em branco no início/fim dos nomes das colunas (boa prática)
df.columns = df.columns.str.strip()

# Lista nome de colunas
print(df.columns)

Index(['data_extracao', 'predial1', 'queda_arr', 'data', 'feridos',
       'feridos_gr', 'mortes', 'morte_post', 'fatais', 'auto', 'taxi',
       'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int', 'caminhao', 'moto',
       'carroca', 'bicicleta', 'outro', 'cont_vit', 'ups', 'patinete',
       'idacidente', 'longitude', 'latitude', 'log1', 'log2', 'tipo_acid',
       'dia_sem', 'hora', 'noite_dia', 'regiao', 'consorcio'],
      dtype='object')


5) Remove colunas 'data_extracao' e 'consorcio'

In [6]:
# Remove colunas 'data_extracao' e 'consorcio'
remover = ['data_extracao', 'consorcio']
existe = [col for col in remover if col in df.columns]

if existe:
    df = df.drop(columns=existe)
else:
    print("Nenhuma das colunas presente")

# Lista nome de colunas depois remoção
print(df.columns)

Index(['predial1', 'queda_arr', 'data', 'feridos', 'feridos_gr', 'mortes',
       'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb',
       'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta',
       'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'longitude',
       'latitude', 'log1', 'log2', 'tipo_acid', 'dia_sem', 'hora', 'noite_dia',
       'regiao'],
      dtype='object')


6) Transforma tipos e remove inválidos

In [7]:
# Mostra tipo de dados, valores não nulos
print("\nInformações gerais do DataFrame:")
df.info()

# Remove inválidos 'data'
df = df.dropna(subset=['data'])

# Converte 'data' para datetime
df['data'] = pd.to_datetime(df['data'], errors='coerce')
print("Coluna 'data' convertida com sucesso.")

# Remove data fora do escopo (2020-2025)
df = df[(df['data'] >= '2020-01-01') & (df['data'] <= '2025-12-31')]
print("Datas fora do escopo (2020-2025) removidas com sucesso.")

# Remove inválidos 'horas'
df = df.dropna(subset=['hora'])

# Cria coluna 'hora_acidente'
df['hora_acidente'] = pd.to_datetime(df['hora'], format='%H:%M:%S.%f', errors='coerce').dt.hour
print("Coluna 'hora' convertida com sucesso.")

# Remove inválidos 'log1', 'regiao'
df = df.dropna(subset=['log1', 'regiao'])

print("\nInformações após:")
df.info()


Informações gerais do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69521 entries, 0 to 69520
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   predial1    65442 non-null  float64
 1   queda_arr   69521 non-null  float64
 2   data        69521 non-null  object 
 3   feridos     69521 non-null  int64  
 4   feridos_gr  69521 non-null  int64  
 5   mortes      69521 non-null  int64  
 6   morte_post  69521 non-null  int64  
 7   fatais      69521 non-null  int64  
 8   auto        69521 non-null  int64  
 9   taxi        69521 non-null  int64  
 10  lotacao     69521 non-null  int64  
 11  onibus_urb  69521 non-null  int64  
 12  onibus_met  69521 non-null  int64  
 13  onibus_int  69521 non-null  int64  
 14  caminhao    69521 non-null  int64  
 15  moto        69521 non-null  int64  
 16  carroca     69521 non-null  int64  
 17  bicicleta   69521 non-null  int64  
 18  outro       69521 non-null  int6

7) Filtra anos 2020-2025 para excluir outliers

In [8]:
# datas em [2020-01-01, 2025-12-31] (seu escopo)
ok_data = df["data"].between("2020-01-01", "2025-12-31") | df["data"].isna()
assert ok_data.all(), "Datas fora do período."

In [9]:
df_20_25 = (
    df.loc[(df["data"] >= "2020-01-01") & (df["data"] <= "2025-04-30")]
      .copy()
)

print(df_20_25.shape)
df_20_25.head()

(68842, 33)


,predial1,queda_arr,data,feridos,feridos_gr,mortes,morte_post,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,longitude,latitude,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,hora_acidente
0,0.0,0.0,2020-10-17,1,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,1,5,0,190816,0.000000,0.000000,R MARCOS MOREIRA,R GASTON ENGLERT,ABALROAMENTO,SÁBADO,19:00:00.0000000,NOITE,NORTE,19
1,598.0,0.0,2020-01-01,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,NaN,NaN,AV BENTO GONCALVES,NaN,ABALROAMENTO,QUARTA-FEIRA,03:00:00.0000000,NOITE,LESTE,3
2,1271.0,0.0,2020-01-01,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0,669097,NaN,NaN,AV INDEPENDENCIA,NaN,ATROPELAMENTO,QUARTA-FEIRA,23:00:00.0000000,NOITE,LESTE,23
3,1901.0,0.0,2020-01-02,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,5,0,669098,NaN,NaN,AV EDUARDO PRADO,NaN,ATROPELAMENTO,QUINTA-FEIRA,00:05:00.0000000,NOITE,SUL,0
4,3302.0,0.0,2020-01-02,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,5,0,669099,-51.211535,-30.081535,AV TERESOPOLIS,NaN,ABALROAMENTO,QUINTA-FEIRA,09:00:00.0000000,DIA,SUL,9


8) Popula Latitude e Longitude ausentes

In [10]:
# Criar df_lat_lon com linhas onde lat ou lon são NaN ou 0
lat = pd.to_numeric(df['latitude'], errors="coerce")
lon = pd.to_numeric(df['longitude'], errors="coerce")

mask = lat.isna() | lon.isna() | (lat == 0) | (lon == 0)
df_lat_lon = df.loc[mask].copy()

df_lat_lon.info()


<class 'pandas.core.frame.DataFrame'>
Index: 10823 entries, 0 to 69519
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   predial1       9422 non-null   float64       
 1   queda_arr      10823 non-null  float64       
 2   data           10823 non-null  datetime64[ns]
 3   feridos        10823 non-null  int64         
 4   feridos_gr     10823 non-null  int64         
 5   mortes         10823 non-null  int64         
 6   morte_post     10823 non-null  int64         
 7   fatais         10823 non-null  int64         
 8   auto           10823 non-null  int64         
 9   taxi           10823 non-null  int64         
 10  lotacao        10823 non-null  int64         
 11  onibus_urb     10823 non-null  int64         
 12  onibus_met     10823 non-null  int64         
 13  onibus_int     10823 non-null  int64         
 14  caminhao       10823 non-null  int64         
 15  moto           10823 non